In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.decomposition as decomposition
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
df_train = pd.read_csv("data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

n_class = np.unique(Y).size

X.shape, Y.shape

((87554, 187, 1), (87554,))

### Parameter search on RNN

In [23]:
def build_gru(n_class=5, dropout=0.3, rnn_sizes = [128, 128], fc_sizes=[64], batch_norm=True):
    nclass = 5
    model = Sequential()
    model.add(Input(shape=(187, 1)))
    
    if batch_norm:
        model.add(BatchNormalization())
        
    for index, dim in enumerate(rnn_sizes):
        model.add(GRU(dim, dropout=dropout, return_sequences=(index != len(rnn_sizes) - 1)))
        
        if batch_norm:
            model.add(BatchNormalization())
    
    for index, dim in enumerate(fc_sizes):
        model.add(Dense(dim, dropout=dropout, activation="relu"))
        
        if batch_norm:
            model.add(BatchNormalization())
            
    model.add(Dense(nclass, activation="softmax"))

    return model

In [24]:
class CustomRNN(BaseEstimator):
    #def __init__():
    #    super().__init__()
    
    def fit(self, train_X, train_y, **kwargs):
        
        self.build_model()
        
        early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
        redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
        callbacks_list = [checkpoint, early, redonplat]  # early
        self.model.fit(train_X, train_y, validation_split=0.1, epochs=self.epochs, batch_size=self.batch_size)
    
    def predict(self, eval_X):
        
        return self.model.predict(eval_X)
    
    def set_params(self, epochs=100, 
                         batch_size=64, 
                         learning_rate=1e-3, 
                         dropout=0.3, 
                         rnn_sizes=[128, 128],
                         fc_sizes=[64],
                         batch_norm=True):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.dropout = dropout
        self.rnn_sizes = rnn_sizes
        self.fc_sizes = fc_sizes
        self.batch_norm = batch_norm
        
        return self
        
    def build_model(self):
        self.model = build_gru(n_class=5, dropout=self.dropout, 
                          rnn_sizes=self.rnn_sizes, fc_sizes=self.fc_sizes, batch_norm=self.batch_norm)
        opt = optimizers.Adam(self.learning_rate)
            
        self.model.compile(optimizer=opt, 
                      loss="sparse_categorical_crossentropy", 
                      metrics=['accuracy'])        

In [25]:
params = {
    'epochs': [100],
    'batch_size': [64],
    'learning_rate': [1e-3],
    'dropout': [0.2],
    'rnn_sizes': [[128, 128], [128, 128, 128], [128, 128, 128, 128], [256, 256], [256, 256, 256], [256, 256, 128]],
    'fc_sizes': [[64], [64, 64], [64, 32]],
    'batch_norm': [True, False]
}

dummy_params = {
    'epochs': [1],
    'batch_size': [16],
    'learning_rate': [1e-3],
    'dropout': [0.2],
    'rnn_sizes': [[64]],
    'fc_sizes': [[64]],
    'batch_norm': [True]
}

model = CustomRNN()
search = GridSearchCV(estimator=model, 
                      param_grid=dummy_params,
                      scoring=make_scorer(f1_score),
                      n_jobs=1, 
                      return_train_score=True, 
                      refit=True, 
                      verbose=10)
best = search.fit(X, Y)
best.__dict__

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64] 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64], score=(train=nan, test=nan), total=   0.2s
[CV] batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64] 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64], score=(train=nan, test=nan), total=   0.2s
[CV] batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64] 


...

[CV]  batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64], score=(train=nan, test=nan), total=   0.2s
[CV] batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64] 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s


[CV]  batch_norm=True, batch_size=16, dropout=0.2, epochs=1, fc_sizes=[64], learning_rate=0.001, rnn_sizes=[64], score=(train=nan, test=nan), total=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.2s finished


{'scoring': make_scorer(f1_score),
 'estimator': CustomRNN(),
 'n_jobs': 1,
 'iid': 'deprecated',
 'refit': False,
 'cv': None,
 'verbose': 10,
 'pre_dispatch': '2*n_jobs',
 'error_score': nan,
 'return_train_score': True,
 'param_grid': {'epochs': [1],
  'batch_size': [16],
  'learning_rate': [0.001],
  'dropout': [0.2],
  'rnn_sizes': [[64]],
  'fc_sizes': [[64]],
  'batch_norm': [True]},
 'multimetric_': False,
 'best_index_': 0,
 'best_score_': nan,
 'best_params_': {'batch_norm': True,
  'batch_size': 16,
  'dropout': 0.2,
  'epochs': 1,
  'fc_sizes': [64],
  'learning_rate': 0.001,
  'rnn_sizes': [64]},
 'scorer_': make_scorer(f1_score),
 'cv_results_': {'mean_fit_time': array([0.22329526]),
  'std_fit_time': array([0.00708975]),
  'mean_score_time': array([0.]),
  'std_score_time': array([0.]),
  'param_batch_norm': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_batch_size': masked_array(data=[16],
          

In [15]:
seq_len = 187

def get_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(128, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=True,
                   dropout=0.2))
    model.add(LSTM(128, 
                   activation='relu',
                   return_sequences=False, 
                   dropout=0.1))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

def get_simple_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(32, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=False,
                   dropout=0.2))
    
    model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

In [16]:
model = get_simple_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 165       
Total params: 5,573
Trainable params: 5,573
Non-trainable params: 0
_________________________________________________________________


In [10]:
mini_X = X[:1000, :, :]
mini_Y = Y[:1000]

np.unique(mini_Y, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int8),
 array([848,  17,  58,  10,  67], dtype=int64))

In [17]:
model.fit(X[:1000, :, :], Y[:1000], 
          epochs=2, 
          verbose=1,
          batch_size=32,
          validation_split=0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/2
900/900 [==============================] - 13s 15ms/sample - loss: 1.5165 - accuracy: 0.8489 - val_loss: 1.3513 - val_accuracy: 0.8300
Epoch 2/2
900/900 [==============================] - 10s 11ms/sample - loss: nan - accuracy: 0.8500 - val_loss: nan - val_accuracy: 0.8300


In [11]:
model.predict(mini_X)

array([[0.305817  , 0.15159464, 0.1644598 , 0.19375268, 0.18437594],
       [0.30581692, 0.15159465, 0.16445978, 0.19375265, 0.18437594],
       [0.30581677, 0.15159471, 0.16445987, 0.19375265, 0.18437597],
       ...,
       [0.30581686, 0.15159468, 0.16445982, 0.19375265, 0.18437594],
       [0.3058147 , 0.1515957 , 0.16446076, 0.19375263, 0.18437621],
       [0.30581692, 0.15159465, 0.16445978, 0.19375265, 0.18437594]],
      dtype=float32)